# Affaan's Bitcoin Price Predictor using Prophet for Time Series

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import tensorflow as tf

In [ ]:
# Read 5 Year Price/Time Daily Increment Data for Bitcoin into a DataFrame
bitcoin = bitcoin = pd.read_csv('BTC5Y.csv')
bitcoin.head()

In [ ]:
bitcoin.info()

In [ ]:
# Convert the "Date" column to a datetime object
bitcoin['Date'] = pd.to_datetime(bitcoin['Date'])

# Set the float format to show up to 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

# Plot a scatter plot of Bitcoin price data with time in years on the x-axis and price on the y-axis
plt.plot(bitcoin['Date'], bitcoin['Close'])

# Add a title and labels to the plot
plt.title('Bitcoin Price Scatter Plot')
plt.xlabel('Year')
plt.ylabel('Price')

# Show the plot
plt.show()

In [ ]:
# Train/test split
split_date = '2021-01-01'
train = bitcoin.loc[bitcoin.index <= split_date]
test = bitcoin.loc[bitcoin.index > split_date]

# Initialize model parameters
smoothing_level = 0.2
smoothing_slope = 0.05
seasonal_period = 365

# Set up cross-validation using TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Fit model and make predictions using cross-validation
rmse_scores = []
for train_index, test_index in tscv.split(train):
    # Train/validation split for cross-validation
    cv_train, cv_val = train.iloc[train_index], train.iloc[test_index]
    
    # Fit model to training data
    model = ExponentialSmoothing(cv_train, trend='add', seasonal='add', seasonal_periods=seasonal_period)
    fitted_model = model.fit(smoothing_level=smoothing_level, smoothing_slope=smoothing_slope)
    
    # Make predictions on validation data
    preds = fitted_model.forecast(len(cv_val))
    
    # Calculate RMSE for validation period
    rmse = np.sqrt(mean_squared_error(cv_val, preds))
    rmse_scores.append(rmse)

# Print average RMSE score
print('Cross-validation RMSE scores:', rmse_scores)
print('Average RMSE score:', np.mean(rmse_scores))

# Fit model to entire training set and make predictions on test set
model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=seasonal_period)
fitted_model = model.fit(smoothing_level=smoothing_level, smoothing_slope=smoothing_slope)
preds = fitted_model.forecast(len(test))

# Plot actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.plot(train.index, train, label='Training set')
plt.plot(test.index, test, label='Test set')
plt.plot(preds.index, preds, label='Predictions')
plt.legend()
plt.title('Bitcoin Price - Actual vs. Predicted (Holt\'s Linear Exponential Smoothing)')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.show()
